In [6]:
import sys
import sys
sys.path.append('../build/')

import numpy as np
np.set_printoptions(precision=6, suppress=True)

import versor as vsr

# PK8103 Swarm Intelligence

This notebook implements the swarm intelligence algorithms **Ant Colony Optimization (ACO)**, **Artificial Bee Colony Optimization (ABC)** and **Particle Swarm Optimization (PSO)** for estimation of **rigid body motion motors** from point clouds using **Conformal Geometric Algebra (CGA)**.

## Conformal Geometric Algebra

### Polar Decomposition

$$M = \frac{X}{|X|}\left(1 - \frac{\langle X \widetilde{X} \rangle_4}{2 \langle X \widetilde{X} \rangle} \right),\ X \in \mathbb{M}$$

In [14]:
def polar_decomposition(x):
    xx = x * x.rev()
    sinv = (vsr.Sca(1.0) + (vsr.Drt(- xx[7] / (2 * xx[0]))))
    return x * sinv

### Cost Function

$$ C = \sum_i^n \| \mathcal{K}^{-1}(b_i) - \mathcal{K}^{-1}(M a_i \widetilde{M})    \|^2 $$

In [15]:
def cost_function(m):
    mot = vsr.Mot(*m)
    mot = polar_decomposition(mot)
    b_ = [point.spin(mot) for point in points_a]
    b = points_b
    return np.sum([np.linalg.norm(pa.to_array()[:3] - 
                                  pb.to_array()[:3]) for pa, pb in zip(b, b_)])

### Dataset generation

In [16]:
n = 10
motor = vsr.Trs.from_vector(vsr.Vec(1,1,1)) * vsr.Rot.from_bivector(vsr.Biv(0,1,0) * np.pi/6.0)
points_a = [vsr.Vec(*np.random.normal(0.0, 0.8, 3)).null() for i in range(n)]
points_b = [point.spin(motor) for point in points_a]

## Artificial Bee Colony Optimization *(ABC)*

## Particle Swarm Optimization *(PSO)*

## Ant Colony Optimization for Continuous Domains *(ACO$_\mathbb{R}$)*